In [ ]:
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
sweep_config = {

    'method' : 'grid',
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'
                },
    'parameters' : {
        'batch_size': {'values': [8,16]},
        'learning_rate': {'values': [0.001,0.0001]},
        'hidden_nodes': {'values': [128,64]},
        'img_size': {'values': [16, 224]},
        'epochs': {'values': [5,10]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-new-sep-7")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: k9k26vi7
Sweep URL: https://wandb.ai/pritkudale-vizuara/5-flowers-new-sep-7/sweeps/k9k26vi7


In [ ]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(config.hidden_nodes, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5)]
      )

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: p47ata3t with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: vizuara-info (pritkudale-vizuara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
    413/Unknown 81s 192ms/step - accuracy: 0.3416 - loss: 1.4702

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 91s 215ms/step - accuracy: 0.3417 - loss: 1.4700 - val_accuracy: 0.4405 - val_loss: 1.2473
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 83s 200ms/step - accuracy: 0.4499 - loss: 1.2542 - val_accuracy: 0.4649 - val_loss: 1.2096
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 92s 222ms/step - accuracy: 0.4955 - loss: 1.1903 - val_accuracy: 0.4622 - val_loss: 1.2125
Epoch 4/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 140s 339ms/step - accuracy: 0.5162 - loss: 1.1436 - val_accuracy: 0.4730 - val_loss: 1.2155
Epoch 5/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 339ms/step - accuracy: 0.5408 - loss: 1.0995 - val_accuracy: 0.4784 - val_loss: 1.2091


...

wandb: Agent Starting Run: aco19pwu with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 85s 203ms/step - accuracy: 0.3300 - loss: 1.5115 - val_accuracy: 0.4378 - val_loss: 1.2919
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 85s 205ms/step - accuracy: 0.4428 - loss: 1.2769 - val_accuracy: 0.4622 - val_loss: 1.2301
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 206ms/step - accuracy: 0.4689 - loss: 1.2131 - val_accuracy: 0.4784 - val_loss: 1.2171
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 341ms/step - accuracy: 0.4843 - loss: 1.1846 - val_accuracy: 0.4784 - val_loss: 1.2211
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 342ms/step - accuracy: 0.5055 - loss: 1.1432 - val_accuracy: 0.5000 - val_loss: 1.1931
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 143s 344ms/step - accuracy: 0.5347 - loss: 1.1117 - val_accuracy: 0.5000 - val_loss: 1.2001
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 164s 253ms/step - accuracy: 0.5513 - loss: 1.0779 - val_accuracy: 0.5027 - val_loss: 1.2016
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 96s 231ms/step - accuracy: 0.5625 - lo

batch/accuracy,▁▁▂▂▃▃▃▃▆▄▅▅▅▅▅▆▆▆▆▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇██
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▆▆▆▆▆▆▆▇▇▇▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▆▆▆▄▄▄▄▃▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch/accuracy,▁▃▄▅▅▆▇▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▁▁
epoch/val_accuracy,▁▄▅▅███▇▆▇
epoch/val_loss,█▄▃▃▁▁▂▂▃▄
batch/accuracy,0.60493


wandb: Agent Starting Run: 2v36g6fk with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 84s 200ms/step - accuracy: 0.2569 - loss: 8.2773 - val_accuracy: 0.2351 - val_loss: 1.6074
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 344ms/step - accuracy: 0.2397 - loss: 1.6040 - val_accuracy: 0.2351 - val_loss: 1.6076
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 84s 204ms/step - accuracy: 0.2397 - loss: 1.6020 - val_accuracy: 0.2351 - val_loss: 1.6085
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 137s 332ms/step - accuracy: 0.2397 - loss: 1.6016 - val_accuracy: 0.2351 - val_loss: 1.6093
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 143s 346ms/step - accuracy: 0.2397 - loss: 1.6015 - val_accuracy: 0.2351 - val_loss: 1.6099
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 143s 346ms/step - accuracy: 0.2397 - loss: 1.6016 - val_accuracy: 0.2351 - val_loss: 1.6103
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 343ms/step - accuracy: 0.2397 - loss: 1.6016 - val_accuracy: 0.2351 - val_loss: 1.6105
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 344ms/step - accuracy: 0.2397 - l